In [1]:
# DO NOT modify this cell.
filename = "Buchtel.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

# Yes, you can create your own test cases and you should. Do it in a new cell.

In [79]:
import numpy as np

def load_pgm_as_numpy_array(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Check if the file is in PGM format
    if lines[0].strip() != 'P2':
        raise ValueError("File is not in PGM format (P2)")

    # Extract image dimensions and maximum pixel value
    width, height = map(int, lines[2].split())
    max_value = int(lines[3])

    # Extract pixel values from the remaining lines
    pixel_values = []
    for line in lines[4:]:
        pixel_values.extend(map(int, line.split()))

    # Convert pixel values to a numpy array
    image_np = np.array(pixel_values).reshape(height, width)

    return image_np, max_value

def calculate_energy(image):
    """Calculate energy of each pixel in the image with edge handling."""

    # Pad the array symmetrically
    padded_arr = np.pad(image, pad_width=1, mode='edge')

    # Create the final padded array with the desired shape
    m, n = image.shape
    padded_image = np.zeros((m+2, n+2), dtype=image.dtype)

    # Fill the interior of the padded array with values from the original array
    padded_image[1:-1, 1:-1] = image

    # Fill the boundary of the padded array with edge values
    padded_image[0, 1:-1] = padded_arr[0, 1:-1]  # top row
    padded_image[-1, 1:-1] = padded_arr[-1, 1:-1]  # bottom row
    padded_image[1:-1, 0] = padded_arr[1:-1, 0]  # left column
    padded_image[1:-1, -1] = padded_arr[1:-1, -1]  # right column

    # Compute gradients in x and y directions
    dx = padded_image[1:-1, 2:] - padded_image[1:-1, :-2]  # Difference along x-axis
    dy = padded_image[2:, 1:-1] - padded_image[:-2, 1:-1]  # Difference along y-axis

    # Calculate energy as the sum of absolute differences in x and y directions
    energy = np.abs(dx) + np.abs(dy)

    return energy

def find_vertical_seam(energy):
    """Find the vertical seam with the lowest energy using dynamic programming."""
    rows, cols = energy.shape
    seam = []

    # Initialize DP table for cumulative minimum energy
    dp = np.zeros_like(energy)
    dp[0] = energy[0]

    # Compute minimum energy seam path
    for i in range(1, rows):
        for j in range(cols):
            # Find the minimum cumulative energy from the top
            if j == 0:
                dp[i, j] = energy[i, j] + min(dp[i-1, j], dp[i-1, j+1])
            elif j == cols - 1:
                dp[i, j] = energy[i, j] + min(dp[i-1, j-1], dp[i-1, j])
            else:
                dp[i, j] = energy[i, j] + min(dp[i-1, j-1], dp[i-1, j], dp[i-1, j+1])

    # Trace back the path of the minimum seam
    min_energy_index = np.argmin(dp[-1])
    seam.append(min_energy_index)

    for i in range(rows - 2, -1, -1):
        prev_index = seam[-1]
        if prev_index == 0:
            next_index = np.argmin(dp[i, prev_index:prev_index+2])
        else:
            next_index = np.argmin(dp[i, prev_index-1:prev_index+2]) + prev_index - 1
        seam.append(next_index)

    seam.reverse()
    return seam

def remove_vertical_seam(image, seam):
    """Remove the vertical seam from the image."""
    rows, cols = image.shape
    new_image = np.zeros((rows, cols - 1), dtype=image.dtype)

    for i in range(rows):
        # Create a mask to remove the seam pixel
        mask = np.ones(cols, dtype=bool)
        mask[seam[i]] = False
        new_image[i, :] = image[i, mask]

    return new_image

def seam_carve(image, num_seams):
    """Perform seam carving on the image."""
    for _ in range(num_seams):
        energy = calculate_energy(image)
        seam = find_vertical_seam(energy)
        image = remove_vertical_seam(image, seam)

    return image


In [80]:
import os

def save_pgm(image_filepath, image_np, max_value, v, h):
    """Save a numpy array as a PGM file."""
    base_filename, ext = os.path.splitext(os.path.basename(image_filepath))
    output_directory = 'test/myProcessed/'
    output_filename = f"{output_directory}{base_filename}_processed_{v}_{h}.pgm"

    with open(output_filename, 'w') as file:
        # Write PGM header
        file.write("P2\n")
        file.write(f"# Created by seam carving \n")
        file.write(f"{image_np.shape[1]} {image_np.shape[0]}\n")  # Width and height
        file.write(f"{max_value}\n")  # Maximum pixel value

        # Write pixel values
        for row in image_np:
            file.write(" ".join(map(str, row)) + "\n")

    print(f"Processed image saved as: {output_filename}")

def process_image(image_filepath, num_seams):
    """Process the image by seam carving and save the result."""
    image_np, max_value = load_pgm_as_numpy_array(image_filepath)
    processed_image = seam_carve(image_np, num_seams)  # Perform seam carving

    save_pgm(image_filepath, processed_image, max_value, num_seams, 0)  # Save the processed image

# User inputs
# image_filename = input("Enter the image file name: ")
# num_seams = int(input("Enter the number of vertical seams to remove: "))

# For testing
image_filepath = './test/raw/CAS.pgm'
vertical_seams = 20

# Entry Point for Part I
process_image(image_filepath, vertical_seams)


376
376
376
376
376
376
376
376
376
376
376
376
376
376
376
376
376
376
376
376
Processed image saved as: test/myProcessed/CAS_processed_20_0.pgm


In [30]:
# Test cell

my = "./test/myProcessed/CAS_processed_20_0.pgm"
correct = "./test/processed/CAS_processed_20_0.pgm"

import filecmp
filecmp.cmp(my, correct)

False

In [3]:
# Part II : both vertical and horizontal seams removal
# Save your processed file to img_processed_v_h.pgm
filename2 = filename.split(".")[0]+"_processed_"+ \
            str(verticalSeam2Remove)+"_"+str(horizontalSeam2Remove)+".pgm"
print(filename2)

# your code, add cells as you need

Buchtel_processed_10_8.pgm


In [4]:
# This is for graduate students only
filenameG = "flower.ppm"
vSeam2Remove = 20
hSeam2Remove = 25

In [5]:
# This is for graduate students only
# Part III : color img in ppm
filenameG2 = filenameG.split(".")[0]+"_processed_"+ \
            str(vSeam2Remove)+"_"+str(hSeam2Remove)+".ppm"
print(filenameG2)
# your code, add cells as you need

flower_processed_20_25.ppm


In [71]:
def calculate_energy(image):
    """Calculate energy of each pixel in the image with edge handling."""

    # Pad the array symmetrically
    padded_arr = np.pad(image, pad_width=1, mode='edge')

    # Create the final padded array with the desired shape
    m, n = image.shape
    padded_image = np.zeros((m+2, n+2), dtype=image.dtype)

    # Fill the interior of the padded array with values from the original array
    padded_image[1:-1, 1:-1] = image

    # Fill the boundary of the padded array with edge values
    padded_image[0, 1:-1] = padded_arr[0, 1:-1]  # top row
    padded_image[-1, 1:-1] = padded_arr[-1, 1:-1]  # bottom row
    padded_image[1:-1, 0] = padded_arr[1:-1, 0]  # left column
    padded_image[1:-1, -1] = padded_arr[1:-1, -1]  # right column

    # Compute gradients in x and y directions
    dx = padded_image[1:-1, 2:] - padded_image[1:-1, :-2]  # Difference along x-axis
    dy = padded_image[2:, 1:-1] - padded_image[:-2, 1:-1]  # Difference along y-axis

    print(padded_image)
    print(dx)
    print(dy)

    # Calculate energy as the sum of absolute differences in x and y directions
    energy = np.abs(dx) + np.abs(dy)

    return energy

# Example usage:
image = np.array([[2, 3],
                  [4, 5]])

energy = calculate_energy(image)
print(energy)

[[0 2 3 0]
 [2 2 3 3]
 [4 4 5 5]
 [0 4 5 0]]
[[1 1]
 [1 1]]
[[2 2]
 [2 2]]
[[3 3]
 [3 3]]
